In [87]:
%%bash
mkdir /python/output

In [124]:
# general
import os
import random
import csv

# wolfram alpha
import wolframalpha

# images
import urllib.request

# email
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

In [125]:
# get list from csv

with open('./input/emails.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
data

FileNotFoundError: [Errno 2] No such file or directory: './input/emails.csv'

Source:<br>
https://www.geeksforgeeks.org/python-create-a-simple-assistant-using-wolfram-alpha-api/

In [84]:
gbl_app_id = '2H79RA-9HVG6K889K' # improvement: get from secrets.py
gbl_str_path_output = "./output/" # end with last /

In [98]:
# randomize
gbl_str_gender = random.choice(['boys','girls'])
gbl_int_rank = random.randint(1,10000)

In [15]:
wc = wolframalpha.Client(gbl_app_id)

In [100]:
# testing
question = " ".join(["#" + str(gbl_int_rank), gbl_str_gender, "name"]) # ie "#1 boys name"
res = wc.query(question)

In [56]:
# subpods are just dicts:
# "All objects returned are dictionary subclasses, so to find out which attributes Wolfram|Alpha has supplied, simply invoke .keys() on the object. Attributes formed from XML attributes can be accessed with or without their “@” prefix (added by xmltodict)."
for pod in res.pods:
    for sub in pod.subpods:
        print(sub.keys())

dict_keys(['@title', 'img', 'plaintext'])
dict_keys(['@title', 'datasources', 'img', 'plaintext'])
dict_keys(['@title', 'microsources', 'datasources', 'img', 'plaintext'])
dict_keys(['@title', 'microsources', 'datasources', 'img', 'plaintext'])
dict_keys(['@title', 'microsources', 'img', 'plaintext'])
dict_keys(['@title', 'microsources', 'img', 'plaintext'])
dict_keys(['@title', 'microsources', 'img', 'plaintext'])
dict_keys(['@title', 'microsources', 'img', 'plaintext'])
dict_keys(['@title', 'microsources', 'datasources', 'img', 'plaintext'])
dict_keys(['@title', 'microsources', 'datasources', 'img', 'plaintext'])


In [59]:
# locations of info
int_pod = 0
for pod in res.pods:
    int_sub = 0
    for sub in pod.subpods:
        print(int_pod, int_sub, sub['plaintext'])
            
        int_sub = int_sub + 1
        print()
    
    int_pod = int_pod + 1
    
# no pods have more than one sub

0 0 top US male given names | by current naming frequency

1 0 Liam
(19837 people per year)

2 0 rank | 1st
fraction | 1 in 91 people (1.1%)
number | 19837 people per year
(US data based on 2018 births and other SSA registrations in the US)

3 0 None

4 0 expected total number alive today | 59122 people
expected population fraction | 1 in 4449 people (0.022%)
expected rank | 726th
most common age | 2 years
(using standard US mortality data)

5 0 None

6 0 Bill | Billy | Bilwa | Bud | Gul | Will | William | Willie | Will'm | Willy | W'm | Wm

7 0 Liam Neeson (actor) (born 1952) (age: 68 years)
Liam Hemsworth (actor) (born 1990) (age: 30 years)
Liam Gallagher (musician) (born 1972) (age: 48 years)
Liam Howlett (musician) (born 1971) (age: 49 years)
Liam Aiken (actor) (born 1990) (age: 30 years)
Liam James (actor) (born 1996) (age: 24 years)
Liam Fox (politician) (born 1961) (age: 59 years)
Liam O'Flaherty (novelist) (1896 to 1984)
Liam Boyce (soccer player) (born 1991) (age: 29 years)
Li

In [101]:
# save raw data
dict_raw = {}
int_pod = 0
for pod in res.pods:
    dict_raw[int_pod] = {}
    sub = next(pod.subpods) # there's only one
    if sub['plaintext'] is not None:
        dict_raw[int_pod] = sub['plaintext'].split('\n')
    else:
        dict_raw[int_pod] = {'@src': sub['img']['@src'], '@alt': sub['img']['@alt']}
    
    int_pod = int_pod + 1
    
dict_raw

{0: ['1644th US male given names | by current naming frequency'],
 1: ['Teddy', '(98 people per year)'],
 2: ['rank | 1639th',
  'fraction | 1 in 18371 people (0.0054%)',
  'number | 98 people per year',
  '(US data based on 2018 births and other SSA registrations in the US)'],
 3: {'@src': 'https://www4c.wolframalpha.com/Calculate/MSP/MSP84621197b29e0da512e800002ca29h869hi8d6cg?MSPStoreType=image/gif&s=16',
  '@alt': 'History for US births Fraction'},
 4: ['expected total number alive today | 24466 people',
  'expected population fraction | 1 in 10751 people (0.0093%)',
  'expected rank | 1245th',
  'most common age | 49 years',
  '(using standard US mortality data)'],
 5: {'@src': 'https://www4c.wolframalpha.com/Calculate/MSP/MSP84641197b29e0da512e800002ah17a49b0c1874i?MSPStoreType=image/gif&s=16',
  '@alt': 'Estimated current age distribution'},
 6: ["Ed | Eddie | Eddy | Edm | Edm'd | Edmond | Edmund | Edrus | Eduardo | Edw | Edward | Edw'd | Tad | Ted | Theo | Theodore | Theotric"]

In [105]:
# downlaod images

str_path_img_history = os.path.join(gbl_str_path_output, str(gbl_int_rank).zfill(5) + " " + dict_raw[1][0] + "-history.jpg")
str_path_img_age = os.path.join(gbl_str_path_output, str(gbl_int_rank).zfill(5) + " " + dict_raw[1][0] + "-age.jpg")

urllib.request.urlretrieve(dict_raw[3]['@src'], str_path_img_history)
urllib.request.urlretrieve(dict_raw[5]['@src'], str_path_img_age)

('./output/01644 Teddy-age.jpg', <http.client.HTTPMessage at 0x7f7bf89fb2b0>)

In [122]:
# send email
# https://realpython.com/python-send-email/

sender_email = "carbobot3030@gmail.com"
# receiver_email = ["ccarbonell228@gmail.com", "katiemoore1010@gmail.com"]
receiver_email = ["ccarbonell228@gmail.com"] # improvement: get from shared volume so we can take her off

message = MIMEMultipart("alternative")
message["Subject"] = "Baby Name: " + dict_raw[1][0]
message["From"] = sender_email
message["To"] = ", ".join(receiver_email)

text = """
""" + dict_raw[1][0] + """
""" + dict_raw[8][0] + """
"""

html = """\
<html>
  <head></head>
  <body>
    <h1>""" + dict_raw[1][0] + """</h1>
    <b style='color:blue'><i>""" + dict_raw[8][0] + """</i></b>
    
    <h2>Random Stats</h2>
    <ul>
        <li>""" + dict_raw[2][0].split("|")[1].strip() + """ most popular</li>
        <li>""" + dict_raw[2][1].split("|")[1].strip() + """</li>
        <li>""" + dict_raw[2][2].split("|")[1].strip() + """</li>
        <li>""" + dict_raw[4][0].replace("|","=") + """</li>
        <li>""" + dict_raw[4][3].replace("|","=") + """</li>
    </ul>
    
    <h2>""" + dict_raw[3]['@alt'] + """</h2>
    <img src="cid:image1">
    
    <h2>""" + dict_raw[5]['@alt'] + """</h2>
    <img src="cid:image2">
    
    <h2>Famous Examples</h2>
    <ul>
    """ + "\n".join(["<li>" + str_example + "</li>" for str_example in dict_raw[7]]) + """
    </ul>
    
    <h2>Sources</h2>
    <ul>
        <li>""" + dict_raw[2][3][1:-1] + """</li>
        <li>""" + dict_raw[4][4][1:-1] + """</li>
    </ul>
  </body>
</html>
"""

# Record the MIME types of both parts - text/plain and text/html.
part1 = MIMEText(text, 'plain')
part2 = MIMEText(html, 'html')

# Attach parts into message container.
# According to RFC 2046, the last part of a multipart message, in this case
# the HTML message, is best and preferred.
message.attach(part1)
message.attach(part2)

# download, encode, and attach image

fp = open(str_path_img_history, 'rb')
msgImage_history = MIMEImage(fp.read())
fp.close()

fp = open(str_path_img_age, 'rb')
msgImage_age = MIMEImage(fp.read())
fp.close()

# Define the image's ID as referenced above

msgImage_history.add_header('Content-ID', '<image1>')
message.attach(msgImage_history)

msgImage_age.add_header('Content-ID', '<image2>')
message.attach(msgImage_age)

# connect to gmail
# for this to work, we need to turn ON less secure app access

port = 465  # For SSL
password = "delthefunkyhomosapien" # improvement: get from secrets

# Create a secure SSL context
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )